In [1]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import utils
import json
import os
import git
import warnings
warnings.filterwarnings("ignore")

In [2]:
from dotenv import load_dotenv
load_dotenv()
HOPSWORKS_API_KEY = os.getenv("HOPSWORKS_API_KEY")
os.environ["HOPSWORKS_API_KEY"] = HOPSWORKS_API_KEY

## Get the necessary data from Hopsworks

In [3]:
project = hopsworks.login()
fs = project.get_feature_store() 
# secrets = utils.secrets_api(project.name)

CITY = "dublin"
STATION = "HEUSTON BRIDGE (NORTH)"

# latitude =
# longitude =

today = datetime.datetime.now()

2025-01-25 08:27:52,953 INFO: Initializing external client


2025-01-25 08:27:52,953 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-01-25 08:27:55,772 INFO: Python Engine initialized.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1207494


In [4]:
# Retrieve feature groups
bike_fg = fs.get_feature_group(
    name='bike_data',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather_data',
    version=1,
)


## Clone and pull the repository with the bike data

In [5]:
# Configuration
REPO_URL = "https://github.com/MaxHalford/bike-sharing-history"
CLONE_DIR = "./bike_data/bike-sharing-history"
TARGET_CITY = "dublin"
FILE_NAME = "jcdecaux.geojson"

In [6]:
if not os.path.exists(CLONE_DIR):
    print("Cloning repository...")
    git.Repo.clone_from(REPO_URL, CLONE_DIR)
repo = git.Repo(CLONE_DIR)

# go to main and pull the latest changes
repo.git.checkout("main", force=True)
repo.remotes.origin.pull()

## Get the latest datetime present in the bike data

In [7]:
# last_bike_datetime = "2025-01-06 15:06:11 UTC"

bike_df = bike_fg.read()

last_bike_datetime = bike_df["datetime"].max()
last_bike_datetime = last_bike_datetime.strftime("%Y-%m-%d %H:%M:%S %Z")

last_bike_datetime

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.10s) 


'2025-01-24 00:00:00 UTC'

## Loop through the commits and convert the bike data into a dataframe

In [8]:
results = {}

# Populate the results dict with the stations
data_file = os.path.join(CLONE_DIR, "data/stations/" + TARGET_CITY + "/" + FILE_NAME)
if os.path.exists(data_file):
    with open(data_file, "r") as f:
        data = f.read()
        data = json.loads(data)
        for feature in data["features"]:
            name = feature["properties"]["name"]
            results[name] = []

In [9]:
start_time = datetime.datetime.now()
last_day_and_hour = None

for commit in repo.iter_commits():
    # Stop when we reach the earliest bike date
    if commit.committed_datetime <= datetime.datetime.strptime(last_bike_datetime, "%Y-%m-%d %H:%M:%S %Z").replace(tzinfo=datetime.timezone.utc):
        print("breaking at: ", commit.committed_datetime)
        break

    # Skip commits from today
    if commit.committed_datetime > today.replace(tzinfo=datetime.timezone.utc).replace(hour=0, minute=0, second=0, microsecond=0):
        first_commited_datetime = repo.commit().committed_datetime
        continue

    day_and_hour = commit.committed_datetime.replace(minute=0, second=0, microsecond=0)
    # print("day_and_hour: ", day_and_hour, " - last_day_and_hour: ", last_day_and_hour)
    if day_and_hour == last_day_and_hour:
        continue
    last_day_and_hour = day_and_hour

    # Get the data for the commit
    print("Processing commit: ", commit.hexsha, " - ", commit.committed_datetime)
    try:
        repo.git.checkout(commit.hexsha, force=True)
    except Exception as e:
        print("Error checking out commit: ", e)
        break

    data_file = os.path.join(CLONE_DIR, "data/stations/" + TARGET_CITY + "/" + FILE_NAME)
    if os.path.exists(data_file):
        with open(data_file, "r") as f:
            data = f.read()
            data = json.loads(data)
            for feature in data["features"]:
                try:
                    results[feature["properties"]["name"]].append([feature["properties"]["available_bikes"], commit.committed_datetime])
                except KeyError:
                    continue

print(results)


Processing commit:  89c0df690feb79e4fd64e108668eced862377c99  -  2025-01-24 23:49:14+00:00


Processing commit:  4d4540c5d3741b1dcd99213f5ba16f31c90a9aef  -  2025-01-24 22:49:01+00:00


Processing commit:  ecf1c6a7e76ac73910291e6c562ffcb307dc1d98  -  2025-01-24 21:49:10+00:00


Processing commit:  8007a1428ad5e2c6df247d7805fd65bce0cc058e  -  2025-01-24 20:49:09+00:00


Processing commit:  79164f25114d42e8a3955cb46100ddfc1691f2d7  -  2025-01-24 19:49:11+00:00


...

Processing commit:  b90231e1a0984c755000d310e1a776f08bb739c9  -  2025-01-24 03:49:14+00:00


Processing commit:  9888fcc27cecf84c75812539020019dbbf2b5936  -  2025-01-24 02:56:55+00:00


Processing commit:  9ab3dbc2233f4b922b2b6e2890fda139771c72b9  -  2025-01-24 01:33:21+00:00


Processing commit:  bedce5f455527dd15033c216402f5913b6b101d0  -  2025-01-24 00:49:48+00:00


breaking at:  2025-01-23 23:49:03+00:00
{'CLARENDON ROW': [[3, datetime.datetime(2025, 1, 24, 23, 49, 14, tzinfo=<git.objects.util.tzoffset object at 0x0000016B87CA0760>)], [4, datetime.datetime(2025, 1, 24, 22, 49, 1, tzinfo=<git.objects.util.tzoffset object at 0x0000016B87CA22F0>)], [7, datetime.datetime(2025, 1, 24, 21, 49, 10, tzinfo=<git.objects.util.tzoffset object at 0x0000016B87CA0790>)], [3, datetime.datetime(2025, 1, 24, 20, 49, 9, tzinfo=<git.objects.util.tzoffset object at 0x0000016B87CA0700>)], [4, datetime.datetime(2025, 1, 24, 19, 49, 11, tzinfo=<git.objects.util.tzoffset object at 0x0000016B87CB6470>)], [1, datetime.datetime(2025, 1, 24, 18, 49, 7, tzinfo=<git.objects.util.tzoffset object at 0x0000016B87CBBA30>)], [1, datetime.datetime(2025, 1, 24, 17, 49, 8, tzinfo=<git.objects.util.tzoffset object at 0x0000016B87DF9030>)], [0, datetime.datetime(2025, 1, 24, 16, 49, 10, tzinfo=<git.objects.util.tzoffset object at 0x0000016B87E025F0>)], [0, datetime.datetime(2025, 1, 24

In [10]:
# turn results into a dataframe
df_bike_today = pd.DataFrame()

for station, values in results.items():
    if len(values) > 0:
        df = pd.DataFrame(values, columns=["num_bikes_available", "datetime"])
        df["datetime"] = pd.to_datetime(df["datetime"], utc=True)
        df["station"] = station.replace(" ", "_")
        df_bike_today = pd.concat([df_bike_today, df])

# if empty, do nothing
if not df_bike_today.empty:
    df_bike_today.dropna(inplace=True)
    df_bike_today["num_bikes_available"] = df_bike_today["num_bikes_available"].astype("float32")
    df_bike_today = df_bike_today[df_bike_today['station'].isin([STATION.replace(" ", "_")])]
    df_bike_today['datetime'] = pd.to_datetime(df_bike_today['datetime'], utc=True).dt.floor('H') + datetime.timedelta(hours=1)
df_bike_today


,num_bikes_available,datetime,station
0,38.0,2025-01-25 00:00:00+00:00,HEUSTON_BRIDGE_(NORTH)
1,37.0,2025-01-24 23:00:00+00:00,HEUSTON_BRIDGE_(NORTH)
2,37.0,2025-01-24 22:00:00+00:00,HEUSTON_BRIDGE_(NORTH)
3,38.0,2025-01-24 21:00:00+00:00,HEUSTON_BRIDGE_(NORTH)
4,40.0,2025-01-24 20:00:00+00:00,HEUSTON_BRIDGE_(NORTH)
5,40.0,2025-01-24 19:00:00+00:00,HEUSTON_BRIDGE_(NORTH)
6,40.0,2025-01-24 18:00:00+00:00,HEUSTON_BRIDGE_(NORTH)
7,40.0,2025-01-24 17:00:00+00:00,HEUSTON_BRIDGE_(NORTH)
8,40.0,2025-01-24 16:00:00+00:00,HEUSTON_BRIDGE_(NORTH)
9,40.0,2025-01-24 15:00:00+00:00,HEUSTON_BRIDGE_(NORTH)


## Fetch the weather data for the same time period

In [11]:
forecast_df = utils.get_hourly_weather_forecast(CITY)
forecast_df = forecast_df.rename(columns={'date_x': 'datetime'})
forecast_df = forecast_df.drop(columns=['date_y', 'date_only'])
forecast_df.dropna(inplace=True)

print(forecast_df.empty)

forecast_df

features: {'hourly': ['temperature_2m', 'apparent_temperature', 'rain', 'snowfall', 'wind_speed_10m'], 'daily': ['daylight_duration', 'rain_sum']}
params: {'latitude': 53.35, 'longitude': -6.26, 'hourly': ['temperature_2m', 'apparent_temperature', 'rain', 'snowfall', 'wind_speed_10m'], 'daily': ['daylight_duration', 'rain_sum']}


Coordinates 53.5°N -6.25°E
Elevation 11.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
False


,datetime,temperature_2m,apparent_temperature,rain,snowfall,wind_speed_10m,daylight_duration,rain_sum,city
0,2025-01-25 00:00:00+00:00,2.20,-3.840192,0.0,0.0,24.735432,30829.769531,0.6,dublin
1,2025-01-25 01:00:00+00:00,1.80,-4.076790,0.0,0.0,23.263912,30829.769531,0.6,dublin
2,2025-01-25 02:00:00+00:00,1.40,-4.220479,0.0,0.0,21.325253,30829.769531,0.6,dublin
3,2025-01-25 03:00:00+00:00,1.10,-4.340653,0.0,0.0,19.995398,30829.769531,0.6,dublin
4,2025-01-25 04:00:00+00:00,1.05,-4.370267,0.0,0.0,19.813087,30829.769531,0.6,dublin
...,...,...,...,...,...,...,...,...,...
235,2025-02-03 19:00:00+00:00,5.55,1.923627,0.0,0.0,15.815435,32731.849609,0.0,dublin
236,2025-02-03 20:00:00+00:00,5.65,1.972600,0.0,0.0,16.087610,32731.849609,0.0,dublin
237,2025-02-03 21:00:00+00:00,5.75,2.020761,0.0,0.0,16.363178,32731.849609,0.0,dublin
238,2025-02-03 22:00:00+00:00,5.75,2.021965,0.0,0.0,16.135872,32731.849609,0.0,dublin


## Insert the bike and weather data into Hopsworks

In [12]:
if df_bike_today.empty:
    print("No bike data available for today")
else:
    bike_fg.insert(df_bike_today)

Uploading Dataframe: 0.00% |                                      | Rows 0/24 | Elapsed Time: 00:00 | Remaining Time: ?

Uploading Dataframe: 4.17% |█▍                                | Rows 1/24 | Elapsed Time: 00:01 | Remaining Time: 00:31

Uploading Dataframe: 100.00% |███████████████████████████████| Rows 24/24 | Elapsed Time: 00:01 | Remaining Time: 00:00

Launching job: bike_data_1_offline_fg_materialization


Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1207494/jobs/named/bike_data_1_offline_fg_materialization/executions


In [13]:
if forecast_df.empty:
    print("No weather forecast available for today")
else:
    weather_fg.insert(forecast_df, write_options={"wait_for_job": True})


Uploading Dataframe: 0.00% |                                     | Rows 0/240 | Elapsed Time: 00:00 | Remaining Time: ?

Uploading Dataframe: 100.00% |█████████████████████████████| Rows 240/240 | Elapsed Time: 00:00 | Remaining Time: 00:00

Launching job: weather_data_1_offline_fg_materialization


Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1207494/jobs/named/weather_data_1_offline_fg_materialization/executions


2025-01-25 08:29:27,668 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED


2025-01-25 08:29:30,877 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED


2025-01-25 08:31:22,784 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED


2025-01-25 08:31:22,952 INFO: Waiting for log aggregation to finish.


2025-01-25 08:31:48,283 INFO: Execution finished successfully.
